In [ ]:
## Code for testing the results of arXiv:2106.09021

In [ ]:
from Less_parameters_functions import train_model
import os
from tqdm import tqdm

In [ ]:
# Testing framework

In [ ]:
current_dir = os.getcwd()
n_list = [50, 100, 150, 200, 300]
layer_types = ['Spectral_full', 'Dense', 'SSVD_full', 'QR']
trials = 3

In [ ]:
configuration = {'n2': None,
                 'save_path': os.path.join(current_dir, 'Results'),
                 'type': None,
                 'dataset': 'MNIST',
                 'learning_rate': 0.015,
                 'batch_size': 200,
                 'epochs': 90,
                 'hidden_layers': 2}

for n in n_list:
    for lay_type in layer_types:
        configuration['n2'] = n
        configuration['type'] = lay_type
        print('\n{} hidden layers of size {}\n'.format(configuration['hidden_layers'], n ))

        for t in tqdm(range(trials)):
            train_model(config=configuration)